In [1]:
import xarray as xr
from pathlib import Path
from CMIP6_IO import CMIP6_IO
import os

## Upload files to be used for weigh calculation
This script will loop over all files found in a speficic folder `lightpath`and find files of the same variable and scenario to be used for weight calculation (ClimWIP). 
The various files are currently focusing on `tos` files that are to be compared to observations (`CORA, ERSST.V5`). All the files are stored in one folder
to be read by ClimWIP.

In [2]:
# Define what models, scenarios, and variables to extract for weight creation

source_ids = ["CanESM5", "MPI-ESM1-2-LR", "UKESM1-0-LL", "MPI-ESM1-2-HR"]
scenarios = ["ssp245"]
var_names = ["tos"]
lightpath = "light"
io = CMIP6_IO()
max_models=3

In [7]:
counter = 0

for var_name in var_names:
    for scenario in scenarios:
        for source_id in source_ids:
            weights_path = f"{lightpath}/weights_test/{var_name}"
            if not os.path.exists(weights_path):
                os.makedirs(weights_path, exist_ok=True)
            current = f"{lightpath}/{scenario}/{source_id}"
            file_on_gcs = io.list_dataset_on_gs(current)
           
            for f in file_on_gcs:
                # Filter to only use the models we prefer (UKESM1-0-LL excluded)
                if any(source_id in f for source_id in source_ids) and var_name in f and "weights" not in f:
                    ds = io.open_dataset_on_gs(f)
                    
                    # Store the cut out region as new file and upload to GCS
                    filename = f"{weights_path}/{f.split('/')[-1][0:-3]}_clipped.nc"
                    ds.to_netcdf(filename)
                    io.upload_to_gcs(filename)
                    counter+=1
                    if counter >= max_models:
                        break
                    # Delete the local version of the file
                    if Path(filename).exists():
                        Path(filename).unlink()
            
                    print(f"Created clipped file {filename}")
                    ds.close()

[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p1f1_ssp245_tos.nc
Opening tos - light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p1f1_ssp245_tos.nc <xarray.Dataset> Size: 233MB
Dimensions:  (time: 1800, lon: 360, lat: 90, y: 90, x: 360)
Coordinates:
  * time     (time) datetime64[ns] 14kB 1950-01-16 1950-02-16 ... 2099-12-16
  * lon      (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat      (lat) float64 720B 0.5 1.5 2.5 3.5 4.5 ... 85.5 86.5 87.5 88.5 89.5
Dimensions without coordinates: y, x
Data variables:
    tos      (time, y, x) float32 233MB ...
Created clipped file light/weights/tos/CMIP6_CanESM5_r10i1p1f1_ssp245_tos_clipped.nc
<xarray.Dataset> Size: 233MB
Dimensions:  (time: 1800, lon: 30, lat: 10, y: 90, x: 360)
Coordinates:
  * time     (time) datetime64[ns] 14kB 1950-01-16 1950-02-16 ... 2099-12-16
  * lon      (lon) float64 240B 5.5 6.5 7.5 8.5 9.5 ... 30.5 31.5 32.5 33.5 34.5
  * lat      (lat) float64 80B 69.5 70.5 71.5 72.5 